In [ ]:
# Import libraries

import requests
import json
import pandas as pd
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm
from openai import OpenAI

### Load data

In [ ]:
# Load documents with ids

doc_url = "https://raw.githubusercontent.com/PerisN/LLMs-Course-DataTalksClub/main/3.%20vector-search/evaluation/documents-with-ids.json"
documents = requests.get(doc_url).json()

In [ ]:
documents[4]

In [ ]:
# Load ground truth data csv, filter and convert into a list of dictionaries

ground_truth_url = "https://raw.githubusercontent.com/PerisN/LLMs-Course-DataTalksClub/main/3.%20vector-search/evaluation/ground-truth-data.csv"
ground_truth = pd.read_csv(ground_truth_url).query("course == 'machine-learning-zoomcamp'").to_dict(orient='records')

In [ ]:
ground_truth[4]

In [ ]:
doc_idx = {d['id']: d for d in documents}
doc_idx['0227b872']['text']

### Index data

In [ ]:
# Initialize a connection to Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_text_vector": {   # use the question and text combination vector since it has the best results
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

In [ ]:
# Define the model to use for embeddings

model_name = 'multi-qa-MiniLM-L6-cos-v1'
model = SentenceTransformer(model_name)

In [ ]:
for doc in tqdm(documents):
    question = doc['question']
    text = doc['text']
    doc['question_text_vector'] = model.encode(question + ' ' + text)

    es_client.index(index=index_name, document=doc)

### Retrieval

In [ ]:
def elastic_search_knn(field, vector, course):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [ ]:
def question_text_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn('question_text_vector', v_q, course)

In [ ]:
question_text_vector_knn(dict(
    question='Are sessions recorded if I miss one?',
    course='machine-learning-zoomcamp'
))

### RAG Flow

In [ ]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [ ]:
client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [ ]:
def llm(prompt):
    response = client.chat.completions.create(
        model='phi3',      
        messages=[{"role": "user", "content": prompt}] 
    )
    
    return response.choices[0].message.content

In [ ]:

def rag(query: dict, model='phi3') -> str:
    search_results = question_text_vector_knn(query)
    prompt = build_prompt(query['question'], search_results)
    answer = llm(prompt)
    return answer

In [ ]:
ground_truth[88]

In [ ]:
rag(ground_truth[88])

In [ ]:
doc_idx['0a278fb2']['text']